In [3]:
import pandas as pd
import numpy as np
from sklearn.metrics import accuracy_score
import seaborn as sns
from sklearn.model_selection import train_test_split, GridSearchCV, cross_val_score
from sklearn.linear_model import LogisticRegression, Ridge
from sklearn.ensemble import RandomForestClassifier, GradientBoostingRegressor, VotingClassifier, StackingClassifier
from sklearn.multioutput import MultiOutputClassifier
from sklearn.linear_model import RidgeClassifier
import xgboost as xgb
import re
import sklearn
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics import classification_report

Pre-Processing

In [5]:
df = pd.read_csv("C:/Users/akash/Documents/GitHub/threepc/systango-hackathon/essays.csv", encoding="cp1252")
df.columns
#print(sklearn.__version__)


1.6.1


In [6]:
!pip install --upgrade scikit-learn

In [7]:
def clean_text(text):
    text = re.sub(r'[^a-zA-Z\s]', '', text)
    text = text.lower()
    return text

In [8]:
df['cleaned_text'] = df['TEXT'].apply(clean_text)
vectorizer = TfidfVectorizer(max_features=5000)
X = vectorizer.fit_transform(df['cleaned_text'])
pd.set_option('future.no_silent_downcasting', True)
y = df[['openness', 'conscientiousness', 'extraversion', 'agreeableness', 'neuroticism']].replace({'y': 1, 'n': 0})
print(y.head())
y = y.fillna(0)

  openness conscientiousness extraversion agreeableness neuroticism
0        1                 0            0             1           1
1        0                 0            0             1           0
2        1                 1            0             0           1
3        0                 1            1             1           0
4        1                 0            1             1           0


In [9]:
ohe_cols = ['openness', 'conscientiousness', 'extraversion', 'agreeableness', 'neuroticism']

After Pre-Processing

In [14]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [15]:
print(np.unique(y_train))
print(y_train.shape)
y_train = y_train.astype(int)

[0 1]
(1973, 5)


In [16]:
rf = RandomForestClassifier(n_estimators=100, random_state=42)
multi_target_rf = MultiOutputClassifier(rf, n_jobs=-1)
multi_target_rf.fit(X_train, y_train)
y_pred = multi_target_rf.predict(X_test)

In [17]:
y_pred = multi_target_rf.predict(X_test)

In [18]:
print(y_test.shape)
print(y_pred.shape)  
y_test = y_test.astype(int)
y_pred = y_pred.astype(int)
print("Labels in y_test:", np.unique(y_test))
print("Labels in y_pred:", np.unique(y_pred))

(494, 5)
(494, 5)
Labels in y_test: [0 1]
Labels in y_pred: [0 1]


In [19]:
print(classification_report(y_test, y_pred, zero_division=1))

              precision    recall  f1-score   support

           0       0.61      0.56      0.59       257
           1       0.55      0.50      0.52       267
           2       0.59      0.62      0.60       267
           3       0.56      0.64      0.59       274
           4       0.50      0.54      0.52       234

   micro avg       0.56      0.57      0.57      1299
   macro avg       0.56      0.57      0.57      1299
weighted avg       0.56      0.57      0.57      1299
 samples avg       0.58      0.60      0.53      1299



In [20]:
df.describe()

,TEXT,extraversion,neuroticism,agreeableness,conscientiousness,openness,cleaned_text
count,2467,2467,2467,2467,2467,2467,2467
unique,2467,2,2,2,2,2,2467
top,"Well, right now I just woke up from a mid-day ...",y,n,y,y,y,well right now i just woke up from a midday na...
freq,1,1276,1234,1310,1253,1271,1


In [21]:
from sklearn.multioutput import MultiOutputRegressor
from sklearn.linear_model import LogisticRegression
lr = LogisticRegression(max_iter=1000)
multi_output_lr = MultiOutputRegressor(lr)
multi_output_lr.fit(X_train, y_train)

MultiOutputRegressor(estimator=LogisticRegression(max_iter=1000))

In [22]:
y_pred = multi_output_lr.predict(X_test)
print(classification_report(y_test, y_pred, zero_division=1))

              precision    recall  f1-score   support

           0       0.61      0.61      0.61       257
           1       0.60      0.55      0.57       267
           2       0.58      0.58      0.58       267
           3       0.57      0.65      0.61       274
           4       0.57      0.63      0.60       234

   micro avg       0.58      0.60      0.59      1299
   macro avg       0.59      0.60      0.59      1299
weighted avg       0.59      0.60      0.59      1299
 samples avg       0.60      0.61      0.55      1299



In [23]:
xgb_model = xgb.XGBClassifier( eval_metric='logloss')
multi_xgb = MultiOutputClassifier(xgb_model)
multi_xgb.fit(X_train, y_train)

MultiOutputClassifier(estimator=XGBClassifier(base_score=None, booster=None,
                                              callbacks=None,
                                              colsample_bylevel=None,
                                              colsample_bynode=None,
                                              colsample_bytree=None,
                                              device=None,
                                              early_stopping_rounds=None,
                                              enable_categorical=False,
                                              eval_metric='logloss',
                                              feature_types=None, gamma=None,
                                              grow_policy=None,
                                              importance_type=None,
                                              interaction_constraints=None,
                                              learning_rate=None, max_bin=None,
                                              max_cat_threshold=None,
                                              max_cat_to_onehot=None,
                                              max_delta_step=None,
                                              max_depth=None, max_leaves=None,
                                              min_child_weight=None,
                                              missing=nan,
                                              monotone_constraints=None,
                                              multi_strategy=None,
                                              n_estimators=None, n_jobs=None,
                                              num_parallel_tree=None,
                                              random_state=None, ...))

In [24]:
y_pred = multi_xgb.predict(X_test)

print(classification_report(y_test, y_pred, zero_division=1))

              precision    recall  f1-score   support

           0       0.60      0.55      0.57       257
           1       0.56      0.51      0.53       267
           2       0.55      0.52      0.54       267
           3       0.58      0.57      0.58       274
           4       0.52      0.59      0.55       234

   micro avg       0.56      0.55      0.55      1299
   macro avg       0.56      0.55      0.55      1299
weighted avg       0.56      0.55      0.55      1299
 samples avg       0.58      0.56      0.51      1299



In [ ]:
params = {
    "estimator__learning_rate": [0.01, 0.1, 0.3],
    "estimator__n_estimators": [100, 300, 500],
    "estimator__max_depth": [3, 5, 7]
}

grid_search = GridSearchCV(multi_xgb, param_grid=params, cv=3, scoring='f1_macro', verbose=1)
grid_search.fit(X_train, y_train)

# Best parameters
print(grid_search.best_params_)

In [26]:
ridge_multi = MultiOutputClassifier(RidgeClassifier())
ridge_multi.fit(X_train, y_train)
y_pred = ridge_multi.predict(X_test)

In [27]:
print(classification_report(y_test, y_pred, zero_division=1))

              precision    recall  f1-score   support

           0       0.61      0.57      0.59       257
           1       0.61      0.57      0.59       267
           2       0.58      0.55      0.57       267
           3       0.57      0.61      0.59       274
           4       0.55      0.61      0.57       234

   micro avg       0.58      0.58      0.58      1299
   macro avg       0.58      0.58      0.58      1299
weighted avg       0.58      0.58      0.58      1299
 samples avg       0.59      0.59      0.54      1299



In [28]:
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.multioutput import MultiOutputClassifier
from sklearn.model_selection import GridSearchCV
gbc = GradientBoostingClassifier()
multi_gbc = MultiOutputClassifier(gbc)
multi_gbc.fit(X_train, y_train)

MultiOutputClassifier(estimator=GradientBoostingClassifier())

In [29]:
y_pred = multi_gbc.predict(X_test)

In [30]:
print(classification_report(y_test, y_pred, zero_division=1))

              precision    recall  f1-score   support

           0       0.63      0.61      0.62       257
           1       0.55      0.55      0.55       267
           2       0.57      0.59      0.58       267
           3       0.58      0.63      0.60       274
           4       0.49      0.54      0.51       234

   micro avg       0.56      0.59      0.57      1299
   macro avg       0.56      0.59      0.57      1299
weighted avg       0.56      0.59      0.57      1299
 samples avg       0.57      0.60      0.53      1299



In [31]:
essay_try = """
Today was an exhilarating rollercoaster of emotions, packed with adventure, social interactions, and unexpected surprises! As an extrovert, I thrive on energy, and today was no exception. The morning started with a burst of enthusiasm as I joined my friends for a coffee catch-up. Our conversation flowed effortlessly, ranging from deep discussions about personal growth to lighthearted jokes that had us in stitches. 

The highlight of my afternoon was an impromptu networking event. Openness is one of my strongest traits, so I found myself fully engaged, embracing new conversations and perspectives. Meeting new people, exchanging ideas, and learning about different industries fueled my curiosity and left me brimming with inspiration. It felt invigorating to share my thoughts and soak in the knowledge of others.

However, my neurotic tendencies made their presence known as well. While I enjoyed every moment, I couldn’t shake off occasional bouts of overthinking—Did I say the right thing? Did I make a good impression? Still, I reminded myself to focus on the positive energy and the joy of the experience rather than dwelling on self-doubt. 

By evening, I found myself at a lively social gathering, where I danced, laughed, and made unforgettable memories. The thrill of being surrounded by people, feeling connected, and sharing genuine emotions reinforced my love for socializing. Yet, as the night wound down, a wave of exhaustion hit me—an odd combination of satisfaction and lingering anxiety. 

Despite the emotional highs and lows, today was yet another affirmation of my personality. I crave interaction, welcome new experiences, and navigate the emotional tides of life with openness and enthusiasm. Each moment, whether thrilling or nerve-wracking, contributes to my growth, making every day an adventure worth embracing.
"""

In [32]:
clean_text(essay_try)

'\ntoday was an exhilarating rollercoaster of emotions packed with adventure social interactions and unexpected surprises as an extrovert i thrive on energy and today was no exception the morning started with a burst of enthusiasm as i joined my friends for a coffee catchup our conversation flowed effortlessly ranging from deep discussions about personal growth to lighthearted jokes that had us in stitches \n\nthe highlight of my afternoon was an impromptu networking event openness is one of my strongest traits so i found myself fully engaged embracing new conversations and perspectives meeting new people exchanging ideas and learning about different industries fueled my curiosity and left me brimming with inspiration it felt invigorating to share my thoughts and soak in the knowledge of others\n\nhowever my neurotic tendencies made their presence known as well while i enjoyed every moment i couldnt shake off occasional bouts of overthinkingdid i say the right thing did i make a good i

In [33]:
essay_try_vectorized = vectorizer.transform([essay_try])
sic = multi_gbc.predict(essay_try_vectorized)
print(sic)

[[1 0 0 1 0]]


In [34]:
import joblib
joblib.dump(multi_gbc, 'personality_model.pkl')
print("Model saved successfully!")

Model saved successfully!


In [78]:
import tkinter as tk
from tkinter import scrolledtext, messagebox
import joblib

model = joblib.load("personality_model.pkl")
root = tk.Tk()
root.title("Personality Predictor")
text_area = scrolledtext.ScrolledText(root, width=80, height=20)
text_area.pack(pady=10)
def predict_personality():
    essay_text = text_area.get("1.0", tk.END).strip()  # Get text
    if not essay_text:
        messagebox.showerror("Error", "Please enter an essay!")
        return
    clean_text(essay_text)
    essay_text_vectorized = vectorizer.transform([essay_text])  # Transform new data
    prediction = multi_gbc.predict(essay_text_vectorized)  # Predict
    messagebox.showinfo("Prediction", f"Predicted Personality Traits  (OCEAN): \n                                               {prediction}")
submit_button = tk.Button(root, text="Predict", command=predict_personality)
submit_button.pack(pady=5)
root.mainloop()
